## Scenario 1: A single data scientist participating in an ML competition

This scenario demonstrates how an individual data scientist can use MLflow to track machine learning experiments on their local machine. This is a common setup for solo projects, hackathons, or competitions where collaboration and remote access are not required.

### MLflow setup overview
- **Tracking server:** Not used (runs locally, no remote server)
- **Backend store:** Local filesystem (stores experiment metadata in the `mlruns/` folder)
- **Artifacts store:** Local filesystem (stores model files and other artifacts in the same `mlruns/` folder)

With this setup, all experiment runs, parameters, metrics, and artifacts are saved locally. You can explore and compare your experiments using the MLflow UI.

### How to use the MLflow UI
- You can launch the MLflow UI by running `mlflow ui` in your terminal, or by running the provided code cell in this notebook.
- The UI will be available at [http://localhost:5000](http://localhost:5000).
- http://127.0.0.1:5000
- Use the UI to browse experiments, compare runs, and inspect logged models and artifacts.

> **Tip:** This local setup is ideal for learning and prototyping. For team projects or production, you would typically use a remote tracking server and a more robust backend (e.g., a database and cloud storage).

In [9]:
%pip install mlflow
import mlflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///Users/marianasaca/Documents/Documents%20-%20Mariana%E2%80%99s%20MacBook%20Air/MDB%20Term%202/ML%20Ops/ie-mlops-nyc-taxis/03-experiment-tracking/mlruns'


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/marianasaca/Documents/Documents%20-%20Mariana%E2%80%99s%20MacBook%20Air/MDB%20Term%202/ML%20Ops/ie-mlops-nyc-taxis/03-experiment-tracking/mlruns/583326327153127534', creation_time=1764073747842, experiment_id='583326327153127534', last_update_time=1764073747842, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/marianasaca/Documents/Documents%20-%20Mariana%E2%80%99s%20MacBook%20Air/MDB%20Term%202/ML%20Ops/ie-mlops-nyc-taxis/03-experiment-tracking/mlruns/0', creation_time=1764073733512, experiment_id='0', last_update_time=1764073733512, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np
import mlflow
import sklearn
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

mlflow.set_experiment("my-experiment-1")

X, y = load_iris(return_X_y=True)
class_names = load_iris().target_names

models = [
    ("LogisticRegression", LogisticRegression(C=1.0, random_state=42, max_iter=1000, solver="lbfgs")),
    ("RandomForestClassifier", RandomForestClassifier(n_estimators=100, random_state=42))
]

for model_name, model in models:
    with mlflow.start_run() as run:
        mlflow.set_tag("model_type", model_name)
        mlflow.log_param("sklearn_version", sklearn.__version__)
        model.fit(X, y)
        y_pred = model.predict(X)
        acc = accuracy_score(y, y_pred)
        mlflow.log_metric("accuracy", acc)
        # Log confusion matrix as labeled DataFrame
        cm = confusion_matrix(y, y_pred)
        cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
        cm_df.to_csv("confusion_matrix_labeled.csv")
        mlflow.log_artifact("confusion_matrix_labeled.csv")
        # Confusion matrix heatmap as image
        plt.figure(figsize=(5,4))
        sns.heatmap(cm_df, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Confusion Matrix ({model_name})")
        plt.ylabel("True label")
        plt.xlabel("Predicted label")
        plt.tight_layout()
        plt.savefig("confusion_matrix_heatmap.png")
        plt.close()
        mlflow.log_artifact("confusion_matrix_heatmap.png")
        # Provide input_example and use 'name' instead of deprecated 'artifact_path'
        input_example = np.expand_dims(X[0], axis=0)
        mlflow.sklearn.log_model(model, name="models", input_example=input_example)
        mlflow.set_tag("n_classes", len(np.unique(y)))
        mlflow.set_tag("run_time", datetime.datetime.now().isoformat())
        mlflow.set_tag("description", f"{model_name} on Iris dataset")
        print(f"Logged run for {model_name}, accuracy={acc:.3f}")

Logged run for LogisticRegression, accuracy=0.973
Logged run for RandomForestClassifier, accuracy=1.000


In [13]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///Users/marianasaca/Documents/Documents%20-%20Mariana%E2%80%99s%20MacBook%20Air/MDB%20Term%202/ML%20Ops/ie-mlops-nyc-taxis/03-experiment-tracking/mlruns/583326327153127534', creation_time=1764073747842, experiment_id='583326327153127534', last_update_time=1764073747842, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///Users/marianasaca/Documents/Documents%20-%20Mariana%E2%80%99s%20MacBook%20Air/MDB%20Term%202/ML%20Ops/ie-mlops-nyc-taxis/03-experiment-tracking/mlruns/0', creation_time=1764073733512, experiment_id='0', last_update_time=1764073733512, lifecycle_stage='active', name='Default', tags={}>]

In [14]:
# Launch MLflow UI (default: uses local mlruns/ folder)
import subprocess
import sys

# This will run 'mlflow ui' as a background process
subprocess.Popen([sys.executable, '-m', 'mlflow', 'ui'])
print("MLflow UI started. Open http://localhost:5000 in your browser.")

MLflow UI started. Open http://localhost:5000 in your browser.


### Interacting with the model registry

In [15]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [ ]:
from mlflow.exceptions import MlflowException

try:
    client.search_registered_models()
except MlflowException:
    print("It's not possible to access the model registry :(")

/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:256: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, artifact_uri)
/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:285: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)
[MLflow] Security middleware enabled with default settings (localhost-only). To allow connections from other hosts, use --host 0.0.0.0 and configure --allowed-hosts and -

INFO:     127.0.0.1:55500 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55500 - "GET /static-files/static/js/main.380f9aea.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55501 - "GET /static-files/static/css/main.280d6c90.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:55500 - "GET /static-files/static/js/3617.10568100.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55501 - "GET /static-files/static/js/5759.45405231.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55502 - "GET /static-files/static/js/762.335831d3.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55502 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=5&order_by=last_update_time+DESC HTTP/1.1" 200 OK


/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:256: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, artifact_uri)
/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:256: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, artifact_uri)


INFO:     127.0.0.1:55506 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=5&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:55512 - "GET /static-files/static/js/565.222c289f.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55510 - "GET /static-files/static/css/565.fd7db8ca.chunk.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:55511 - "GET /static-files/static/js/6016.a73a7053.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55508 - "GET /static-files/static/js/8799.de860ae0.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55509 - "GET /static-files/static/js/2365.08729b99.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55508 - "POST /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:55508 - "GET /ajax-api/2.0/mlflow/traces?experiment_ids=583326327153127534&order_by=timestamp_ms%20DESC&max_results=1&filter= HTTP/1.1" 200 OK
INFO:     127.0.0.1:55512 - "GET /static-files/static/js/4499.a03329c0.chunk.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:55508 - "GET /static-files/static/js/897

/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:256: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, artifact_uri)


INFO:     127.0.0.1:55509 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55510 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55510 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55509 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55511 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55517 - "GET /ajax-api/2.0/mlflow/traces?experiment_ids=583326327153127534&order_by=timestamp_ms%20DESC&max_results=1&filter= HTTP/1.1" 200 OK
INFO:     127.0.0.1:55518 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55523 - "POST /ajax-api/2.0/mlflow/experiments/set-experiment-tag HTTP/1.1" 200 OK
INFO:     127.0.0.1:55523 - "POST /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:55524 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55527 - "GET /stat

/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:285: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)
/Users/marianasaca/Documents/Documents - Mariana’s MacBook Air/MDB Term 2/ML Ops/ie-mlops-nyc-taxis/.venv/lib/python3.10/site-packages/mlflow/server/handlers.py:285: FutureWarning: Filesystem model registry backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri)


INFO:     127.0.0.1:55525 - "POST /ajax-api/2.0/mlflow/experiments/search-datasets HTTP/1.1" 200 OK
INFO:     127.0.0.1:55525 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55528 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55527 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55534 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55540 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55547 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55556 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55561 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55564 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55568 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     12